# Exercise 10: Mixed effects

1. Loading and formatting the data 1/1
2. Model fitting 3/4: the interaction term is missing
3. Model assessment 4/4
4. Reflection 1/1

This homework assignment is designed to give you practice fitting and interpreting mixed effects models.

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again.

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file.

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**.

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [ ]:
# WRITE YOUR CODE HERE
# setwd("../Homework datasets/lexDat") # Uncomment this when running for the first time
library(tidyverse)

lex <- read.csv('LexicalData_withIncorrect.csv')
item <- read.csv('Items.csv')

lex |> left_join(item[c('Length', 'Log_Freq_HAL', 'Word')],
	by = join_by(D_Word == Word)) |>
	drop_na() ->
	lex

lex$D_RT <- as.numeric(gsub(",", "", lex$D_RT)) # also fix the commas
head(lex)

,X,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore,Correct,Length,Log_Freq_HAL
,<int>,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>
1,1,157,1,1,710,browse,false,-0.437,1,6,8.856
2,2,67,1,1,1094,refrigerant,false,0.825,1,11,4.644
3,3,120,1,1,587,gaining,false,-0.645,1,7,8.304
4,4,21,1,1,984,cheerless,false,0.025,1,9,2.639
5,5,236,1,1,577,pattered,false,-0.763,1,8,1.386
6,6,236,2,1,715,conjures,false,-0.364,1,8,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output.

In [ ]:
# WRITE YOUR CODE HERE
lm.fit = glm(D_RT ~ Length+Log_Freq_HAL, data=lex)

summary(lm.fit)


Call:
glm(formula = D_RT ~ Length + Log_Freq_HAL, data = lex)

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  786.1944     7.6927  102.20   <2e-16 ***
Length        29.0685     0.6311   46.06   <2e-16 ***
Log_Freq_HAL -30.8857     0.6559  -47.09   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 147907.1)

    Null deviance: 1.1435e+10  on 70588  degrees of freedom
Residual deviance: 1.0440e+10  on 70586  degrees of freedom
AIC: 1040643

Number of Fisher Scoring iterations: 2


Now, install `lme4` using `install.packages()` and then load the library.

In [ ]:
# WRITE YOUR CODE HERE
# install.packages("lme4")	# run this once
library(lme4)


Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output.

In [ ]:
# WRITE YOUR CODE HERE
# Random intercepts only
me.fit = lmer(D_RT ~ Length+Log_Freq_HAL + (1 | Sub_ID), data=lex)

summary(me.fit)


Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Length + Log_Freq_HAL + (1 | Sub_ID)
   Data: lex

REML criterion at convergence: 1012480

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.2412 -0.5450 -0.1601  0.3038 11.0720 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 51073    226.0   
 Residual             97260    311.9   
Number of obs: 70589, groups:  Sub_ID, 299

Fixed effects:
             Estimate Std. Error t value
(Intercept)  785.3529    14.4874   54.21
Length        29.6056     0.5127   57.75
Log_Freq_HAL -31.3619     0.5331  -58.83

Correlation of Fixed Effects:
            (Intr) Length
Length      -0.365       
Log_Frq_HAL -0.331  0.355

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why?

> *Write your response here*
>
> The absolute values of t-values in the mixed effect model are larger compared to the fixed effects model (57.75 vs 46.06 for `Length`; -58.83 vs -47.09 for `Log_Freq_HAL`). Thus, we can say that isolating the random effect made the predicted relationship stronger.

Use the Akaike Information Criterion (AIC) to compare these two models. Which one is better?

In [ ]:
# WRITE YOUR CODE HERE
# We will compare the two models using the Akaike information criterion (AIC)
ic = AIC(lm.fit, me.fit)
ic
diff(ic$AIC)


,df,AIC
,<dbl>,<dbl>
lm.fit,4,1040643
me.fit,5,1012490


[1] -28153.39

> *Write your response here*
>
> `me.fit` has the lower AIC, which means that it has a better fit.

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set?


> *Write your response here*
>
> It appears the the `Trial` number can be regarded as a random effect.

**DUE:** 5pm EST, March 18, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*